# Solution — Task 02: Fixtures & Parametrize

## Setup

In [ ]:
import subprocess, sys, os, textwrap, tempfile, pathlib

FIXTURES = os.path.abspath(os.path.join("..", "fixtures", "input"))
if not os.path.exists(FIXTURES):
    FIXTURES = os.path.abspath(os.path.join("fixtures", "input"))
sys.path.insert(0, FIXTURES)

def run_pytest(test_code: str, extra_files: dict[str, str] | None = None, extra_args: list[str] | None = None):
    with tempfile.TemporaryDirectory() as td:
        td_path = pathlib.Path(td)
        p = td_path / "test_tmp.py"
        p.write_text(textwrap.dedent(test_code))
        if extra_files:
            for name, content in extra_files.items():
                (td_path / name).write_text(textwrap.dedent(content))
        cmd = [sys.executable, "-m", "pytest", str(td), "-v", "--tb=short", "--no-header"]
        if extra_args:
            cmd.extend(extra_args)
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        print(result.stdout + result.stderr)
        return result.returncode

print("Setup complete.")

## Solution 2.1: Fixtures in conftest.py

In [ ]:
conftest_code = f'''
import sys, pytest
sys.path.insert(0, "{FIXTURES}")

@pytest.fixture
def sample_texts():
    return [
        "Hello world",
        "  extra   spaces  ",
        "UPPERCASE TEXT",
        "hello\\tworld\\nfoo",
        "simple",
    ]

@pytest.fixture
def empty_text():
    return ""

@pytest.fixture
def text_with_emails():
    return "Contact alice@example.com and bob@test.org for info."
'''

test_code = f'''
import sys
sys.path.insert(0, "{FIXTURES}")
from sample_module import tokenize, extract_emails, clean_text

def test_tokenize_samples(sample_texts):
    for text in sample_texts:
        tokens = tokenize(text)
        assert isinstance(tokens, list)
        if text.strip():
            assert len(tokens) > 0

def test_clean_empty(empty_text):
    assert clean_text(empty_text) == ""

def test_extract_from_email_text(text_with_emails):
    emails = extract_emails(text_with_emails)
    assert len(emails) >= 2
    assert "alice@example.com" in emails
    assert "bob@test.org" in emails
'''

rc = run_pytest(test_code, extra_files={"conftest.py": conftest_code})
assert rc == 0, "Tests should pass"
print("Task 2.1 passed!")

## Solution 2.2: Parametrize ML Preprocessing

In [ ]:
test_preprocess = f'''
import sys, pytest
sys.path.insert(0, "{FIXTURES}")
from sample_ml_pipeline import preprocess_texts

@pytest.mark.parametrize("texts, expected", [
    (["Hello World"], ["hello world"]),
    (["Check https://example.com out"], ["check out"]),
    (["  extra   spaces  "], ["extra spaces"]),
    (["UPPER case Text"], ["upper case text"]),
    (["no change"], ["no change"]),
])
def test_preprocess(texts, expected):
    assert preprocess_texts(texts) == expected

def test_preprocess_multiple():
    texts = ["Hello", "  World  "]
    result = preprocess_texts(texts)
    assert result == ["hello", "world"]
'''

rc = run_pytest(test_preprocess)
assert rc == 0, "Tests should pass"
print("Task 2.2 passed!")

## Solution 2.3: `tmp_path` for File-Based Tests

In [ ]:
test_file = f'''
import sys
sys.path.insert(0, "{FIXTURES}")
import pandas as pd
from sample_module import clean_text

def test_csv_processing(tmp_path):
    csv_path = tmp_path / "data.csv"
    csv_path.write_text("text" + chr(10) + "  hello   world  " + chr(10) + "  foo   bar  " + chr(10))
    df = pd.read_csv(csv_path)
    df["cleaned"] = df["text"].apply(clean_text)
    assert df["cleaned"].tolist() == ["hello world", "foo bar"]

def test_json_roundtrip(tmp_path):
    import json
    data = {{"texts": ["hello", "world"], "labels": [1, 0]}}
    json_path = tmp_path / "data.json"
    json_path.write_text(json.dumps(data))
    loaded = json.loads(json_path.read_text())
    assert loaded == data
'''

rc = run_pytest(test_file)
assert rc == 0, "Tests should pass"
print("Task 2.3 passed!")

## Solution 2.4: Fixture with Yield

In [ ]:
test_yield = f'''
import sys, pytest, numpy as np
sys.path.insert(0, "{FIXTURES}")
from sample_ml_pipeline import create_pipeline, train_pipeline, predict, SAMPLE_TEXTS, SAMPLE_LABELS

cleanup_log = []

@pytest.fixture
def trained_pipe():
    np.random.seed(42)
    pipe = create_pipeline(max_features=100)
    train_pipeline(pipe, SAMPLE_TEXTS, SAMPLE_LABELS)
    yield pipe
    cleanup_log.append("pipeline cleaned up")

def test_predict(trained_pipe):
    preds = predict(trained_pipe, ["great product"])
    assert len(preds) == 1

def test_cleanup_happened():
    assert "pipeline cleaned up" in cleanup_log
'''

rc = run_pytest(test_yield)
assert rc == 0, "Tests should pass"
print("Task 2.4 passed!")